In [1]:
import sklearn as sk
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("train_data.csv")

In [3]:
df1 = df.drop(["Aboutus","Aboutus_Duration","Browser","ID","Operating_Systems"],axis = 1)
df1.dtypes

Item_page                 float64
Item_page_Duration        float64
Checkout_page             float64
Checkout_page_Duration    float64
Bounce_Rates              float64
Exit_Rates                float64
Page_Values               float64
Remarkable_Day            float64
Month                      object
Province                    int64
TrafficType                 int64
VisitorType                object
Weekend                    object
Income                      int64
dtype: object

In [4]:
#Encode
d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
df1.Month = df1.Month.map(d)
df1

,Item_page,Item_page_Duration,Checkout_page,Checkout_page_Duration,Bounce_Rates,Exit_Rates,Page_Values,Remarkable_Day,Month,Province,TrafficType,VisitorType,Weekend,Income
0,1.0,10.0,9.0,700.000000,0.000000,0.011111,29.621890,0.0,12.0,9,10,Returning_Visitor,no,1
1,2.0,15.0,10.0,894.666667,0.000000,0.022222,0.000000,0.0,5.0,4,2,Returning_Visitor,no,0
2,1.0,85.0,14.0,306.500000,0.000000,0.004444,0.000000,0.0,3.0,3,2,Returning_Visitor,no,0
3,5.0,175.1,26.0,615.559524,0.000000,0.002083,62.773672,0.0,9.0,1,3,New_Visitor,1,1
4,2.0,25.0,5.0,40.000000,0.066667,0.083333,0.000000,0.6,5.0,1,1,Returning_Visitor,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9938,0.0,0.0,8.0,376.333333,0.000000,0.028571,53.988000,0.0,5.0,2,4,Returning_Visitor,1,1
9939,1.0,0.0,22.0,671.866667,0.017391,0.020580,0.000000,0.0,11.0,7,10,Returning_Visitor,no,0
9940,0.0,0.0,15.0,872.833333,0.000000,0.026667,0.000000,0.0,3.0,2,3,Returning_Visitor,1,0
9941,2.0,327.5,7.0,570.500000,0.000000,0.028571,0.000000,0.0,11.0,2,2,Returning_Visitor,no,0


In [5]:
df1["VisitorType"] = df1["VisitorType"].astype('category')
df1["Weekend"] = df1["Weekend"].astype('category')
df1.dtypes

Item_page                  float64
Item_page_Duration         float64
Checkout_page              float64
Checkout_page_Duration     float64
Bounce_Rates               float64
Exit_Rates                 float64
Page_Values                float64
Remarkable_Day             float64
Month                      float64
Province                     int64
TrafficType                  int64
VisitorType               category
Weekend                   category
Income                       int64
dtype: object

In [6]:
categorical_data = df1.drop(['Item_page', 'Item_page_Duration', 'Checkout_page','Checkout_page_Duration','Bounce_Rates','Exit_Rates','Page_Values','Remarkable_Day','Month','Province','TrafficType','Income'], axis=1)
categorical_data

,VisitorType,Weekend
0,Returning_Visitor,no
1,Returning_Visitor,no
2,Returning_Visitor,no
3,New_Visitor,1
4,Returning_Visitor,no
...,...,...
9938,Returning_Visitor,1
9939,Returning_Visitor,no
9940,Returning_Visitor,1
9941,Returning_Visitor,no


In [7]:
numeric_data = df1.drop(['VisitorType','Weekend'],axis = 1)
numeric_data

,Item_page,Item_page_Duration,Checkout_page,Checkout_page_Duration,Bounce_Rates,Exit_Rates,Page_Values,Remarkable_Day,Month,Province,TrafficType,Income
0,1.0,10.0,9.0,700.000000,0.000000,0.011111,29.621890,0.0,12.0,9,10,1
1,2.0,15.0,10.0,894.666667,0.000000,0.022222,0.000000,0.0,5.0,4,2,0
2,1.0,85.0,14.0,306.500000,0.000000,0.004444,0.000000,0.0,3.0,3,2,0
3,5.0,175.1,26.0,615.559524,0.000000,0.002083,62.773672,0.0,9.0,1,3,1
4,2.0,25.0,5.0,40.000000,0.066667,0.083333,0.000000,0.6,5.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9938,0.0,0.0,8.0,376.333333,0.000000,0.028571,53.988000,0.0,5.0,2,4,1
9939,1.0,0.0,22.0,671.866667,0.017391,0.020580,0.000000,0.0,11.0,7,10,0
9940,0.0,0.0,15.0,872.833333,0.000000,0.026667,0.000000,0.0,3.0,2,3,0
9941,2.0,327.5,7.0,570.500000,0.000000,0.028571,0.000000,0.0,11.0,2,2,0


In [8]:
categorical_data["VisitorType"] = categorical_data["VisitorType"].cat.codes
categorical_data["Weekend"] = categorical_data["Weekend"].cat.codes
categorical_data["VisitorType"] 

0       2
1       2
2       2
3       0
4       2
       ..
9938    2
9939    2
9940    2
9941    2
9942    2
Name: VisitorType, Length: 9943, dtype: int8

In [9]:
dataset_new=categorical_data.join(numeric_data)
dataset_new.head()

,VisitorType,Weekend,Item_page,Item_page_Duration,Checkout_page,Checkout_page_Duration,Bounce_Rates,Exit_Rates,Page_Values,Remarkable_Day,Month,Province,TrafficType,Income
0,2,1,1.0,10.0,9.0,700.000000,0.000000,0.011111,29.621890,0.0,12.0,9,10,1
1,2,1,2.0,15.0,10.0,894.666667,0.000000,0.022222,0.000000,0.0,5.0,4,2,0
2,2,1,1.0,85.0,14.0,306.500000,0.000000,0.004444,0.000000,0.0,3.0,3,2,0
3,0,0,5.0,175.1,26.0,615.559524,0.000000,0.002083,62.773672,0.0,9.0,1,3,1
4,2,1,2.0,25.0,5.0,40.000000,0.066667,0.083333,0.000000,0.6,5.0,1,1,0


In [10]:
#column_means = df.mean()
#dataSet_final = dataset_new.fillna(column_means)

In [11]:
# dataSet_final.dtypes
dataSet_final = dataset_new.dropna(axis = 0)
dataSet_final.isnull().sum()

VisitorType               0
Weekend                   0
Item_page                 0
Item_page_Duration        0
Checkout_page             0
Checkout_page_Duration    0
Bounce_Rates              0
Exit_Rates                0
Page_Values               0
Remarkable_Day            0
Month                     0
Province                  0
TrafficType               0
Income                    0
dtype: int64

In [12]:
data = dataSet_final.iloc[:,0:13]
data

,VisitorType,Weekend,Item_page,Item_page_Duration,Checkout_page,Checkout_page_Duration,Bounce_Rates,Exit_Rates,Page_Values,Remarkable_Day,Month,Province,TrafficType
0,2,1,1.0,10.0,9.0,700.000000,0.000000,0.011111,29.621890,0.0,12.0,9,10
1,2,1,2.0,15.0,10.0,894.666667,0.000000,0.022222,0.000000,0.0,5.0,4,2
2,2,1,1.0,85.0,14.0,306.500000,0.000000,0.004444,0.000000,0.0,3.0,3,2
3,0,0,5.0,175.1,26.0,615.559524,0.000000,0.002083,62.773672,0.0,9.0,1,3
4,2,1,2.0,25.0,5.0,40.000000,0.066667,0.083333,0.000000,0.6,5.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9938,2,0,0.0,0.0,8.0,376.333333,0.000000,0.028571,53.988000,0.0,5.0,2,4
9939,2,1,1.0,0.0,22.0,671.866667,0.017391,0.020580,0.000000,0.0,11.0,7,10
9940,2,0,0.0,0.0,15.0,872.833333,0.000000,0.026667,0.000000,0.0,3.0,2,3
9941,2,1,2.0,327.5,7.0,570.500000,0.000000,0.028571,0.000000,0.0,11.0,2,2


In [13]:
target = dataSet_final.iloc[:,13]
target

0       1
1       0
2       0
3       1
4       0
       ..
9938    1
9939    0
9940    0
9941    0
9942    0
Name: Income, Length: 9700, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.3)

In [16]:
from sklearn.svm import SVC #importing the SVM
from sklearn.decomposition import PCA #importing PCA
from sklearn.pipeline import make_pipeline


In [17]:
pca=PCA(n_components=2,whiten=True,random_state=42)
svc=SVC(kernel='rbf')

model=make_pipeline(pca,svc)

In [18]:
model.fit(train_data,train_target)

Pipeline(steps=[('pca', PCA(n_components=2, random_state=42, whiten=True)),
                ('svc', SVC())])

In [19]:
predicted_target=model.predict(test_data)
predicted_target

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [20]:
from sklearn.metrics import accuracy_score

acc=accuracy_score(test_target,predicted_target)
print(acc)

0.8395189003436426


In [21]:
##########################################################
# Testing Set
##########################################################

df_test = pd.read_csv("test_data.csv")

In [22]:
ids=df_test['ID'].values


In [23]:
df1_test = df_test.drop(["ID","Aboutus","Aboutus_Duration","Browser","Operating_Systems"],axis = 1)
df1_test.dtypes

Item_page                   int64
Item_page_Duration        float64
Checkout_page               int64
Checkout_page_Duration    float64
Bounce_Rates              float64
Exit_Rates                float64
Page_Values               float64
Remarkable_Day            float64
Month                      object
Province                    int64
TrafficType                 int64
VisitorType                object
Weekend                    object
dtype: object

In [24]:
d1 = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}
df1_test.Month = df1_test.Month.map(d1)
df1_test

,Item_page,Item_page_Duration,Checkout_page,Checkout_page_Duration,Bounce_Rates,Exit_Rates,Page_Values,Remarkable_Day,Month,Province,TrafficType,VisitorType,Weekend
0,12,569.825000,127,6065.016218,0.006716,0.009265,0.152167,0.0,8.0,2,4,Returning_Visitor,no
1,0,0.000000,25,770.028571,0.011200,0.034933,0.000000,0.0,3.0,8,10,Returning_Visitor,no
2,7,38.600000,70,1087.700000,0.000000,0.004861,0.000000,0.0,10.0,4,5,Returning_Visitor,no
3,4,105.266667,35,655.684762,0.000000,0.005128,0.000000,0.0,11.0,7,2,New_Visitor,1
4,0,0.000000,3,63.000000,0.000000,0.066667,0.000000,0.8,5.0,9,1,Returning_Visitor,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0.000000,2,4.000000,0.000000,0.100000,0.000000,0.8,5.0,4,1,Returning_Visitor,no
996,1,43.000000,36,2362.500000,0.000000,0.016190,0.000000,0.6,5.0,1,4,Returning_Visitor,no
997,0,0.000000,25,1017.966667,0.008333,0.017361,0.000000,0.0,8.0,4,1,Returning_Visitor,1
998,0,0.000000,18,587.733333,0.000000,0.022222,32.523808,0.0,11.0,7,4,Returning_Visitor,1


In [25]:
df1_test["VisitorType"] = df1_test["VisitorType"].astype('category')
df1_test["Weekend"] = df1_test["Weekend"].astype('category')
df1_test.dtypes

Item_page                    int64
Item_page_Duration         float64
Checkout_page                int64
Checkout_page_Duration     float64
Bounce_Rates               float64
Exit_Rates                 float64
Page_Values                float64
Remarkable_Day             float64
Month                      float64
Province                     int64
TrafficType                  int64
VisitorType               category
Weekend                   category
dtype: object

In [26]:
categorical_data_test = df1_test.drop(['Item_page', 'Item_page_Duration', 'Checkout_page','Checkout_page_Duration','Bounce_Rates','Exit_Rates','Page_Values','Remarkable_Day','Month','Province','TrafficType'], axis=1)
categorical_data_test

,VisitorType,Weekend
0,Returning_Visitor,no
1,Returning_Visitor,no
2,Returning_Visitor,no
3,New_Visitor,1
4,Returning_Visitor,no
...,...,...
995,Returning_Visitor,no
996,Returning_Visitor,no
997,Returning_Visitor,1
998,Returning_Visitor,1


In [27]:
numeric_data_test = df1_test.drop(['VisitorType','Weekend'],axis = 1)
numeric_data_test

,Item_page,Item_page_Duration,Checkout_page,Checkout_page_Duration,Bounce_Rates,Exit_Rates,Page_Values,Remarkable_Day,Month,Province,TrafficType
0,12,569.825000,127,6065.016218,0.006716,0.009265,0.152167,0.0,8.0,2,4
1,0,0.000000,25,770.028571,0.011200,0.034933,0.000000,0.0,3.0,8,10
2,7,38.600000,70,1087.700000,0.000000,0.004861,0.000000,0.0,10.0,4,5
3,4,105.266667,35,655.684762,0.000000,0.005128,0.000000,0.0,11.0,7,2
4,0,0.000000,3,63.000000,0.000000,0.066667,0.000000,0.8,5.0,9,1
...,...,...,...,...,...,...,...,...,...,...,...
995,0,0.000000,2,4.000000,0.000000,0.100000,0.000000,0.8,5.0,4,1
996,1,43.000000,36,2362.500000,0.000000,0.016190,0.000000,0.6,5.0,1,4
997,0,0.000000,25,1017.966667,0.008333,0.017361,0.000000,0.0,8.0,4,1
998,0,0.000000,18,587.733333,0.000000,0.022222,32.523808,0.0,11.0,7,4


In [28]:
categorical_data_test["VisitorType"] = categorical_data_test["VisitorType"].cat.codes
categorical_data_test["Weekend"] = categorical_data_test["Weekend"].cat.codes


In [29]:
dataset_new_test=categorical_data_test.join(numeric_data_test)
dataset_new_test

,VisitorType,Weekend,Item_page,Item_page_Duration,Checkout_page,Checkout_page_Duration,Bounce_Rates,Exit_Rates,Page_Values,Remarkable_Day,Month,Province,TrafficType
0,2,1,12,569.825000,127,6065.016218,0.006716,0.009265,0.152167,0.0,8.0,2,4
1,2,1,0,0.000000,25,770.028571,0.011200,0.034933,0.000000,0.0,3.0,8,10
2,2,1,7,38.600000,70,1087.700000,0.000000,0.004861,0.000000,0.0,10.0,4,5
3,0,0,4,105.266667,35,655.684762,0.000000,0.005128,0.000000,0.0,11.0,7,2
4,2,1,0,0.000000,3,63.000000,0.000000,0.066667,0.000000,0.8,5.0,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2,1,0,0.000000,2,4.000000,0.000000,0.100000,0.000000,0.8,5.0,4,1
996,2,1,1,43.000000,36,2362.500000,0.000000,0.016190,0.000000,0.6,5.0,1,4
997,2,0,0,0.000000,25,1017.966667,0.008333,0.017361,0.000000,0.0,8.0,4,1
998,2,0,0,0.000000,18,587.733333,0.000000,0.022222,32.523808,0.0,11.0,7,4


In [30]:
dataset_new_test.isnull().sum()

VisitorType                0
Weekend                    0
Item_page                  0
Item_page_Duration         0
Checkout_page              0
Checkout_page_Duration     0
Bounce_Rates               0
Exit_Rates                 0
Page_Values                0
Remarkable_Day             0
Month                     25
Province                   0
TrafficType                0
dtype: int64

In [31]:
dataset_new_test['Month'] = dataset_new_test['Month'].fillna(1.0)
dataset_new_test.isnull().sum()

VisitorType               0
Weekend                   0
Item_page                 0
Item_page_Duration        0
Checkout_page             0
Checkout_page_Duration    0
Bounce_Rates              0
Exit_Rates                0
Page_Values               0
Remarkable_Day            0
Month                     0
Province                  0
TrafficType               0
dtype: int64

In [32]:
predicted_target=model.predict(dataset_new_test) #getting predictions from the model
print(predicted_target)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [33]:
output = pd.DataFrame({'ID': ids, 'Income': predicted_target})

In [34]:
output

,ID,Income
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
995,996,0
996,997,0
997,998,0
998,999,0


In [35]:
output.to_csv("HellRaisers3.csv")